# GPT-SoVITS on Sagemaker

## build image

In [22]:
!chmod +x ./*.sh && ./build_and_push.sh 

Downloading: 100%|█████████▉| 840M/840M [09:24<00:00, 1.56MB/s]
Downloading: 100%|██████████| 19.1k/19.1k [00:00<00:00, 914kB/s]m
Downloading: 100%|██████████| 7.90M/7.90M [00:01<00:00, 8.27MB/s]
Downloading: 100%|██████████| 48.7k/48.7k [00:00<00:00, 1.15MB/s]
Downloading: 100%|██████████| 91.5k/91.5k [00:00<00:00, 2.01MB/s]
2024-08-22 11:00:52,935 - modelscope - INFO - Use user-specified model revision: v2.0.4
Downloading: 100%|██████████| 7.85k/7.85k [00:00<00:00, 5.95MB/s]
Downloading: 100%|██████████| 1.19k/1.19k [00:00<00:00, 994kB/s]m
Downloading: 100%|██████████| 365/365 [00:00<00:00, 302kB/s]91m
Downloading: 100%|██████████| 1.64M/1.64M [00:00<00:00, 8.39MB/s]
Downloading: 100%|██████████| 8.45k/8.45k [00:00<00:00, 6.11MB/s]
Downloading: 100%|██████████| 27.3k/27.3k [00:00<00:00, 1.30MB/s]
Downloading: 100%|██████████| 2.16M/2.16M [00:00<00:00, 12.5MB/s]
2024-08-22 11:01:00,738 - modelscope - INFO - Use user-specified model revision: v2.0.4
Downloading: 100%|██████████| 6.00k/

In [23]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
bucket = sess.default_bucket()
image="gpt-sovits-inference"
s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

full_image_uri=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image}:latest"
print(full_image_uri)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
687912291502.dkr.ecr.us-west-2.amazonaws.com/gpt-sovits-inference:latest


## remote debug deploy test

In [24]:
!pip list|grep -i sagemaker
!pip list|grep -i boto3

sagemaker                      2.188.0
sagemaker-pyspark              1.4.5
sagemaker-ssh-helper           2.1.0
aioboto3                       12.0.0
boto3                          1.34.143


In [25]:
## empty model data for byoc with webserver
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'gpt_sovits')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'gpt_sovits')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-687912291502/gpt_sovits/assets/model.tar.gz


In [26]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
model = Model(image_uri=full_image_uri, model_data=model_data, role=role,dependencies=[SSHModelWrapper.dependency_dir()] )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [27]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
instance_type = "ml.g5.xlarge"
endpoint_name = sagemaker.utils.name_from_base("gpt-sovits-inference")


ssh_wrapper = SSHModelWrapper.create(model, connection_wait_time_seconds=0)  # <--NEW--

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    wait=False
)


#instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=900)  # <--NEW-- 
#print(f"To connect over SSM run: aws ssm start-session --target {instance_ids[0]}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [31]:
instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=100)

In [32]:
instance_ids[0]

'mi-03ad9bcf34c866231'

## Streams test (only for stream branch deployment)

In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt-sovits/wav/speech_20240425104005663.mp3 ./
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715140344.wav ./

In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715150796.wav ./

In [33]:
import requests

chunk_bytes=None

def upsert(lst, new_dict):
    for i, item in enumerate(lst):
        if new_dict['index'] == i:
            lst[i] = new_dict
            return lst
    lst.append(new_dict)
    return lst

def invoke_streams_endpoint(smr_client,endpointName, request):
    global chunk_bytes
    content_type = "application/json"
    payload = json.dumps(request,ensure_ascii=False)

    response_model = smr_client.invoke_endpoint_with_response_stream(
        EndpointName=endpointName,
        ContentType=content_type,
        Body=payload,
    )

    result = []
    print(response_model['ResponseMetadata'])
    event_stream = iter(response_model['Body'])
    index = 0
    try: 
        while True:
            event = next(event_stream)
            eventChunk = event['PayloadPart']['Bytes']
            chunk_dict = {}
            if index == 0:
                print("Received first chunk")
                chunk_dict['first_chunk'] = True
                chunk_dict['bytes'] = eventChunk
                chunk_bytes = eventChunk
                chunk_dict['last_chunk'] = False
                chunk_dict['index'] = index
            else:
                chunk_dict['first_chunk'] = False
                chunk_dict['bytes'] = eventChunk
                chunk_bytes = eventChunk
                chunk_dict['last_chunk'] = False
                chunk_dict['index'] = index
            print("chunk len:",len(chunk_dict['bytes']))
            result.append(chunk_dict)    
            index += 1
            #print('返回chunk：', chunk_dict['bytes'])
    except StopIteration:
        print("All chunks processed")
        chunk_dict = {}
        chunk_dict['first_chunk'] = False
        chunk_dict['bytes'] = chunk_bytes
        chunk_dict['last_chunk'] = True
        chunk_dict['index'] = index-1
        result = upsert(result,chunk_dict)
    print("result",result)
    return result





In [20]:
import json
import boto3
endpointName="gpt-sovits-inference-2024-08-22-11-03-26-674"
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")
#endpointName="gpt-sovits-sagemaker-endpoint2024-04-03-23-49-44"



request = {"refer_wav_path": "s3://sagemaker-us-west-2-687912291502/gpt-sovits/wav/out003.wav",
    "prompt_text": "后来我就在直播间里认识了越来越多的听友，渐渐的这份工作，也为我带来了一些兼职收入，我就决定把这份工作做下去。",
    "prompt_language": "zh",
    "text": "作为SAP基础架构专家和SAP系统管理员,我来解释一下SAP Basis的含义:SAP Basis是指SAP系统的基础设施层,负责管理和维护整个SAP系统环境的运行。它包括以下几个主要方面:SAP系统管理包括SAP系统实例的安装、启动、监控、备份、升级等日常管理任务。Basis团队负责保证系统的正常运行。",
    "text_language": "zh",
    "cut_punc":","}


In [ ]:
response=invoke_streams_endpoint(runtime_sm_client,endpointName,request)

In [21]:
!aws s3 cp  s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1722482391.mp3 ./

download: s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1722482391.mp3 to ./gpt_sovits_1722482391.mp3


In [13]:
import json
import boto3
endpointName="gpt-sovits-inference-2024-07-24-02-47-05-276"
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")
#endpointName="gpt-sovits-sagemaker-endpoint2024-04-03-23-49-44"



request = {"refer_wav_path":"s3://sagemaker-us-west-2-687912291502/gpt-sovits/wav/speech_20240425104005663.mp3",
    "prompt_text": "私はスポーツが好きな女の子で、私は中華料理が大好きで、私は中国へ旅行するのが好きで、特に杭州、成都が好きです",
    "prompt_language":"ja",
    "text":"When I practice my spells in weather like this, I can do half the work for double the impact..",
    "text_language" :"en",
    "output_s3uri":"s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/",
    "cut_punc":"、"}


In [14]:
response=invoke_streams_endpoint(runtime_sm_client,endpointName,request)

{'RequestId': '93427d96-c00c-4b03-b3e1-ecd44fb7a82e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '93427d96-c00c-4b03-b3e1-ecd44fb7a82e', 'x-amzn-invoked-production-variant': 'AllTraffic', 'x-amzn-sagemaker-content-type': 'application/json', 'date': 'Wed, 07 Aug 2024 03:16:50 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive'}, 'RetryAttempts': 0}
Received first chunk
chunk len: 8192
chunk len: 8185
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
chunk len: 8192
ch

In [12]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1718929830.mp3 ./

download: s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1718929830.mp3 to ./gpt_sovits_1718929830.mp3
